# Tensorflow를 활용한 Faster-RCNN 구현

reference : http://solarisailab.com/archives/2422

In [1]:
!pip install tf_slim
!pip install numpy==1.19.5
!pip install tensorflow_io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 10.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.8 MB 22.8 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
tensorflow 2.8.2+zzzcolab20220719082949 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
cmdstanpy 1.0.4 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.9 MB 1.2 MB/s 


In [2]:
!git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 76561, done.
remote: Counting objects: 100% (382/382), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 76561 (delta 179), reused 320 (delta 139), pack-reused 76179
Receiving objects: 100% (76561/76561), 596.77 MiB | 30.56 MiB/s, done.
Resolving deltas: 100% (54290/54290), done.


In [3]:
%cd /content/models/research/object_detection

/content/models/research/object_detection


In [4]:
# Make sure you grab the latest version
!curl -OL https://github.com/google/protobuf/releases/download/v3.2.0/protoc-3.2.0-linux-x86_64.zip

# Unzip
!unzip protoc-3.2.0-linux-x86_64.zip -d protoc3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1299k  100 1299k    0     0  5057k      0 --:--:-- --:--:-- --:--:-- 12.9M
Archive:  protoc-3.2.0-linux-x86_64.zip
   creating: protoc3/include/
   creating: protoc3/include/google/
   creating: protoc3/include/google/protobuf/
  inflating: protoc3/include/google/protobuf/timestamp.proto  
  inflating: protoc3/include/google/protobuf/source_context.proto  
  inflating: protoc3/include/google/protobuf/any.proto  
  inflating: protoc3/include/google/protobuf/api.proto  
  inflating: protoc3/include/google/protobuf/descriptor.proto  
  inflating: protoc3/include/google/protobuf/struct.proto  
  inflating: protoc3/include/google/protobuf/wrappers.proto  
   creating: protoc3/

In [5]:
%cd ..

/content/models/research


protobuf compile

In [6]:
!protoc object_detection/protos/*.proto --python_out=.

In [7]:
# 경로 설정 추가
!export PYTHONPATH=$PYTHONPATH:/content/models/research
!export PYTHONPATH=$PYTHONPATH:/content/models/research/slim

In [8]:
# 데이터 다운로드
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

--2022-08-28 15:38:32--  http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz [following]
--2022-08-28 15:38:32--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz [following]
--2022-08-28 15:38:33--  https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7

In [9]:
!tar -xvf images.tar.gz
!tar -xvf annotations.tar.gz

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
annotations/trimaps/._pomeranian_180.png
annotations/trimaps/pomeranian_180.png
annotations/trimaps/._pomeranian_181.png
annotations/trimaps/pomeranian_181.png
annotations/trimaps/._pomeranian_182.png
annotations/trimaps/pomeranian_182.png
annotations/trimaps/._pomeranian_183.png
annotations/trimaps/pomeranian_183.png
annotations/trimaps/._pomeranian_184.png
annotations/trimaps/pomeranian_184.png
annotations/trimaps/._pomeranian_185.png
annotations/trimaps/pomeranian_185.png
annotations/trimaps/._pomeranian_186.png
annotations/trimaps/pomeranian_186.png
annotations/trimaps/._pomeranian_187.png
annotations/trimaps/pomeranian_187.png
annotations/trimaps/._pomeranian_188.png
annotations/trimaps/pomeranian_188.png
annotations/trimaps/._pomeranian_189.png
annotations/trimaps/pomeranian_189.png
annotations/trimaps/._pomeranian_19.png
annotations/trimaps/pomeranian_19.png
annotations/trimaps/._pomeranian_190.png
annotations/trimaps/pomeranian_190.png
annota

In [10]:
!pwd

/content/models/research


In [11]:
!ls

adversarial_text    cvt_text	   lfads		  rebar
annotations	    deeplab	   lstm_object_detection  seq_flow_lite
annotations.tar.gz  deep_speech    marco		  slim
attention_ocr	    delf	   nst_blogpost		  vid2depth
audioset	    efficient-hrl  object_detection
autoaugment	    images	   pcl_rl
cognitive_planning  images.tar.gz  README.md


In [12]:
!cp /content/models/research/object_detection/dataset_tools/create_pet_tf_record.py create_pet_tf_record.py

In [13]:
!cp /content/models/research/object_detection/legacy/train.py train.py

In [14]:
!cp /content/models/research/object_detection/legacy/eval.py eval.py

In [15]:
# tf_record 데이터 생성
!python create_pet_tf_record.py \
    --label_map_path=object_detection/data/pet_label_map.pbtxt \
    --data_dir='/content/models/research' \
    --output_dir='/content/models/research'

2022-08-28 15:39:16.416483: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
I0828 15:39:16.425098 139690331830144 create_pet_tf_record.py:273] Reading from Pet dataset.
I0828 15:39:16.438645 139690331830144 create_pet_tf_record.py:288] 2576 training and 1104 validation examples.
I0828 15:39:16.439505 139690331830144 create_pet_tf_record.py:241] On image 0 of 2576
/content/models/research/object_detection/utils/dataset_util.py:83: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:
I0828 15:39:16.764025 139690331830144 create_pet_tf_record.py:241] On image 100 of 2576
I0828 15:39:16.999389 139690331830144 create_pet_tf_record.py:241] On image 200 of 2576
I0828 15:39:17.239463 139690331830144 create_pet_tf_record.py:241] On image 300 of 2576
I0828 15:39:17.476833 139690331830144 create_pet_tf_record.py:241]

In [16]:
!wget http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz
!tar -xvf faster_rcnn_resnet101_coco_11_06_2017.tar.gz

--2022-08-28 15:39:25--  http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.81.208, 142.251.163.128, 142.250.188.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.81.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 595490113 (568M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet101_coco_11_06_2017.tar.gz’

faster_rcnn_resnet1 100%[===================>] 567.90M   120MB/s    in 4.9s    

2022-08-28 15:39:30 (117 MB/s) - ‘faster_rcnn_resnet101_coco_11_06_2017.tar.gz’ saved [595490113/595490113]

faster_rcnn_resnet101_coco_11_06_2017/
faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.index
faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.meta
faster_rcnn_resnet101_coco_11_06_2017/frozen_inference_graph.pb
faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.data-00000-of-00001
faster_rcnn_resnet101

아래 폴더를 확인해보자.  
/models/research/faster_rcnn_resnet101_coco_11_06_2017

In [17]:
!mv /content/models/research/pet_faces_train.record-00001-of-00010 /content/models/research/faster_rcnn_resnet101_coco_11_06_2017
!mv /content/models/research/pet_faces_val.record-00001-of-00010 /content/models/research/faster_rcnn_resnet101_coco_11_06_2017

In [18]:
!cp object_detection/data/pet_label_map.pbtxt faster_rcnn_resnet101_coco_11_06_2017/pet_label_map.pbtxt

faster_rcnn_resnet101_coco_11_06_2017 목록  
- frozen_inference_graph.pb  
- graph.pbtxt  
- model.ckpt.data-00000-of-00001  
- model.ckpt.index  
- model.ckpt.meta  
- pet_faces_train.record-00001-of-00010  
- pet_faces_val.record-00001-of-00010  

아래 파일을 열어보자.  
object_detection/samples/configs/faster_rcnn_resnet101_pets.config 

PATH_TO_BE_CONFIGURED 표시가 되어있는 부분들을 찾아  
각 경로로 바꿔주자.

### 모델 학습

In [ ]:
!cp -r /content/models/research/slim/deployment deployment

In [ ]:
!cp -r /content/models/research/slim/nets nets 

In [ ]:
!ls

adversarial_text
annotations
annotations.tar.gz
attention_ocr
audioset
autoaugment
cognitive_planning
create_pet_tf_record.py
cvt_text
deeplab
deep_speech
delf
deployment
efficient-hrl
eval.py
faster_rcnn_resnet101_coco_11_06_2017
faster_rcnn_resnet101_coco_11_06_2017.tar.gz
images
images.tar.gz
lfads
lstm_object_detection
marco
nets
nst_blogpost
object_detection
pcl_rl
pet_faces_train.record-00000-of-00010
pet_faces_train.record-00002-of-00010
pet_faces_train.record-00003-of-00010
pet_faces_train.record-00004-of-00010
pet_faces_train.record-00005-of-00010
pet_faces_train.record-00006-of-00010
pet_faces_train.record-00007-of-00010
pet_faces_train.record-00008-of-00010
pet_faces_train.record-00009-of-00010
pet_faces_val.record-00000-of-00010
pet_faces_val.record-00002-of-00010
pet_faces_val.record-00003-of-00010
pet_faces_val.record-00004-of-00010
pet_faces_val.record-00005-of-00010
pet_faces_val.record-00006-of-00010
pet_faces_val.record-00007-of-00010
pet_faces_val.record-00008-of-000

In [ ]:
!python train.py \
--pipeline_config_path=object_detection/samples/configs/faster_rcnn_resnet101_pets.config \
--train_dir=train_result

Using TensorFlow backend.
Instructions for updating:
Use object_detection/model_main.py.
W0617 04:38:09.272611 140056500361088 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/absl/app.py:258: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
W0617 04:38:09.317627 140056500361088 deprecation.py:323] From /content/models/research/object_detection/legacy/trainer.py:265: create_global_step (from tf_slim.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.create_global_step
Traceback (most recent call last):
  File "train.py", line 186, in <module>
    tf.app.run()
  File "/tensorflow-1.15.2/python3.7/tensorflow_core/python/platform/app.py", line 40, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local

In [ ]:
!python eval.py \
    --pipeline_config_path=object_detection/samples/configs/faster_rcnn_resnet101_pets.config \
    --checkpoint_dir=train_result \
    --eval_dir=eval_result

Using TensorFlow backend.
Traceback (most recent call last):
  File "eval.py", line 52, in <module>
    from object_detection.legacy import evaluator
  File "/content/models/research/object_detection/legacy/evaluator.py", line 24, in <module>
    from object_detection import eval_util
  File "/content/models/research/object_detection/eval_util.py", line 35, in <module>
    from object_detection.metrics import coco_evaluation
  File "/content/models/research/object_detection/metrics/coco_evaluation.py", line 25, in <module>
    from object_detection.metrics import coco_tools
  File "/content/models/research/object_detection/metrics/coco_tools.py", line 51, in <module>
    from pycocotools import coco
  File "/usr/local/lib/python3.7/dist-packages/pycocotools/coco.py", line 52, in <module>
    from . import mask as maskUtils
  File "/usr/local/lib/python3.7/dist-packages/pycocotools/mask.py", line 3, in <module>
    import pycocotools._mask as _mask
  File "pycocotools/_mask.pyx", line 1

In [ ]:
!tensorboard --logdir=eval_result

TensorBoard 1.15.0 at http://3244721fee37:6006/ (Press CTRL+C to quit)
